In [68]:
import pandas as pd
from pathlib import Path
import numpy as np

base_path = Path('../../..').parent.absolute()

yield_data = pd.read_parquet(f'{base_path}/Data/processed/africa_crop_yield_data.parquet')
yield_data.index = pd.to_datetime(yield_data.index, format='%Y').to_period('A')

# yield_data = yield_data.pct_change()
yield_data = yield_data.pct_change()
yield_data = yield_data.replace(0, 0.01)

In [69]:
climate_data = pd.read_parquet(f'{base_path}/Data/processed/climate_full.parquet')

climate_data.index = pd.date_range(start='1901-01-01', end='2100-12-31', freq='A').to_period('A')

climate_data = climate_data.loc[:'2021', :]

country_map = pd.read_csv('../../Data/resources/countries.csv', encoding="latin-1").set_index('alpha-3')

# Do some trickery to turn the columns into a MultiIndex.
climate_data.columns = pd.MultiIndex.from_tuples([[country_map.loc[x[0], 'name'], x[1], x[2], 'raw'] if len(x) == 3 else [country_map.loc[x[0], 'name'], x[1], x[2], x[3]] for x in climate_data.columns.str.split('_').to_list()])

# Convert the country ISO codes to country names.
climate_data = climate_data.groupby(level=[0,2,3], axis=1).first()


In [70]:
climate_data = climate_data.loc[:, climate_data.columns.get_level_values(1).isin(['prstar', 'tasstar', 'tas', 'pr'])]
climate_data.columns = climate_data.columns.droplevel(2)
climate_data

Algeria                                                      Angola  \
             pr          pr     prstar      tas    tas tasstar           pr   
1901        NaN  102.860000        NaN      NaN  22.58     NaN          NaN   
1902        NaN   91.500000        NaN      NaN  22.58     NaN          NaN   
1903        NaN   73.280000        NaN      NaN  22.48     NaN          NaN   
1904        NaN  105.600000        NaN      NaN  22.65     NaN          NaN   
1905        NaN   87.810000        NaN      NaN  22.43     NaN          NaN   
...         ...         ...        ...      ...    ...     ...          ...   
2017  77.878000   68.880000  -8.998000  23.4000  23.60  0.2000   990.397600   
2018  79.017200   88.240000   9.222800  23.4176  23.23 -0.1876   991.102400   
2019  78.664800   70.910000  -7.754800  23.4264  23.38 -0.0464   994.069600   
2020  77.826400   55.420000 -22.406400  23.4360  23.54  0.1040   998.447200   
2021  76.490692   61.767304 -14.723388  23.4764  24.06  0.5836  1002.655788   

                                       ...      Zambia                  \
               pr     prstar      tas  ...      prstar      tas    tas   
1901   970.600000        NaN      NaN  ...         NaN      NaN  21.56   
1902   988.090000        NaN      NaN  ...         NaN      NaN  21.81   
1903  1013.010000        NaN      NaN  ...         NaN      NaN  21.88   
1904   930.550000        NaN      NaN  ...         NaN      NaN  21.28   
1905   929.880000        NaN      NaN  ...         NaN      NaN  21.77   
...           ...        ...      ...  ...         ...      ...    ...   
2017  1005.980000  15.582400  21.8528  ...  127.800400  22.2840  22.21   
2018  1005.530000  14.427600  21.8556  ...   46.292400  22.2888  21.99   
2019  1012.260000  18.190400  21.8608  ...   16.338000  22.2992  22.44   
2020  1019.670000  21.222800  21.8508  ...   28.931200  22.2736  22.07   
2021  1047.114699  44.458911  21.8668  ...   25.503297  22.2868  22.62   

                Zimbabwe                                                 
     tasstar          pr         pr      prstar      tas    tas tasstar  
1901     NaN         NaN  836.85000         NaN      NaN  20.94     NaN  
1902     NaN         NaN  655.70000         NaN      NaN  21.33     NaN  
1903     NaN         NaN  549.50000         NaN      NaN  21.52     NaN  
1904     NaN         NaN  694.97000         NaN      NaN  20.47     NaN  
1905     NaN         NaN  508.35000         NaN      NaN  21.34     NaN  
...      ...         ...        ...         ...      ...    ...     ...  
2017 -0.0740  647.128400  863.07000  215.941600  21.7996  21.65 -0.1496  
2018 -0.2988  644.506400  607.47000  -37.036400  21.8168  22.03  0.2132  
2019  0.1408  653.722400  695.15000   41.427600  21.8552  22.30  0.4448  
2020 -0.2036  661.696800  655.02000   -6.676800  21.8468  21.86  0.0132  
2021  0.3332  658.820077  630.32193  -28.498147  21.8748  22.19  0.3152  

[121 rows x 312 columns]

In [71]:
full_df = climate_data.merge(yield_data, left_index=True, right_index=True).sort_index(axis=1)

df1 = full_df.melt(value_vars=full_df.columns.tolist(), var_name=['Country', 'Variable'],  ignore_index=False).reset_index()
df1['index'] = df1['index'].dt.to_timestamp()
full = df1.pivot_table(index=['Country', 'index'], columns='Variable', values='value')


In [72]:
full['tasstar2'] = full['tasstar'] ** 2
full['prstar2'] = full['prstar'] ** 2

full['tas2']= full['tas'] ** 2
full['pr2'] = full['pr'] ** 2

In [73]:
from linearmodels.panel import PanelOLS
import statsmodels.api as sm

out = pd.DataFrame()
res = {}

errored_vars = 0

for crop in full.columns[~full.columns.isin(['tas', 'tasstar', 'tasstar2', 'pr', 'prstar', 'prstar2', 't1', 'p1', 'tas2', 'pr2'])]:

    try:
        exog_vars = ['tasstar',  'tasstar2', 'prstar', 'prstar2']
        exog = full[exog_vars]

        model = PanelOLS(full[crop], exog, entity_effects=True)
        results = model.fit(cov_type='clustered', cluster_entity=True)

        out_cntry = results.estimated_effects
        res[crop] = results
        
        out_cntry[['tasstar', 'tasstar2', 'prstar', 'prstar2']] = results.params
        
        out_cntry['crop'] = crop
        out_cntry = out_cntry.reset_index().set_index(['Country', 'crop', 'index'])

        out = pd.concat([out, out_cntry], axis=0)
    
    except Exception as e:
        errored_vars += 1
        continue



/Users/jackminchin/opt/anaconda3/lib/python3.8/site-packages/linearmodels/panel/model.py:1184: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/Users/jackminchin/opt/anaconda3/lib/python3.8/site-packages/linearmodels/panel/model.py:1184: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/Users/jackminchin/opt/anaconda3/lib/python3.8/site-packages/linearmodels/panel/model.py:1184: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/Users/jackminchin/opt/anaconda3/lib/python3.8/site-packages/linearmodels/panel/model.py:1184: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init

In [97]:
major_crops = ['Wheat', 'Soybeans', 'Maize', 'Barley', 'Apples', 'Apricots', 'Beans, dry', 'Cabbages and other brassicas', 'Oats', 'Rice, paddy', 'Seed cotton', 'Tangerines, mandarins, clementines, satsumas', 'Tobacco, unmanufactured', 'Tomatoes', 'Watermelons']
regression_table = pd.DataFrame(index=major_crops, columns=exog_vars)

for crop in major_crops:
    results = res[crop]

    parameters = results.params
    pvalues = results.pvalues

    for i, param in parameters.iteritems():

        star = ''

        if pvalues[i] < 0.1:
            star = '*'
        elif pvalues[i] < 0.05:
            star = '**'
        elif pvalues[i] < 0.01:
            star = '***'


        regression_table.loc[crop, i] = f'{param:.4f}{star}'

regression_table.to_clipboard(index=True, header=True)
regression_table.columns = ['T', 'T$^2$', 'P', 'P$^2$' ]


regression_table.to_latex('../output/regression_table.tex', escape=False)

regression_table = regression_table.style.set_properties(**{'text-align': 'right'})



regression_table

/var/folders/z7/kscqgjb16xs6t1mmw0gy5p4m0000gn/T/ipykernel_58620/2220353973.py:28: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  regression_table.to_latex('../output/regression_table.tex', escape=False)


,T,T$^2$,P,"P$^2$ \footnote{T and P denote deviations from the long run trend of temperature and precipitation respectively, T$^2$ and P$^2$ are their squared terms.}"
Wheat,-0.0166,0.0243,0.0002*,-0.0000
Soybeans,0.0040,-0.0987*,0.0000,0.0000
Maize,0.0347,-0.1629*,0.0002*,-0.0000
Barley,0.0188,-0.0103,0.0003*,0.0000
Apples,0.0233,-0.0478*,-0.0000,0.0000
Apricots,-0.0235*,-0.0005,-0.0001,0.0000*
"Beans, dry",0.0306,-0.0509,0.0001*,-0.0000*
Cabbages and other brassicas,0.0045,-0.0168,-0.0000,0.0000
Oats,-0.0813,0.0194,0.0001,0.0000
"Rice, paddy",0.0167,-0.0384,-0.0000,-0.0000*


In [75]:
out = out.sort_index().groupby(level=[0,1], axis=0).first()

In [76]:
out.to_parquet('./damage_coefficients.parquet')

In [77]:
# out = out[out['tas'] < 0]

In [78]:
out.to_parquet('./damage_coefficients.parquet')

In [79]:
out.to_excel('./damage_coefficients.xlsx')